In [1]:
import pandas as pd
import ollama
from openai import OpenAI
import openai
from fireworks.client import Fireworks
import os
from dotenv import load_dotenv
import anthropic 
import json

# Load environment variables from .env file
load_dotenv('.env')

# Get API keys from environment variables
openai_key = os.environ.get("OPENAI_KEY")
anthropic_key = os.environ.get("ANTHROPIC_API_KEY")
fireworks_key = os.environ.get("FIREWORKS_KEY")

# Initialize OpenAI, Anthropics, and Fireworks clients with respective API keys
client = OpenAI(api_key=openai_key)
anthropic_client = anthropic.Anthropic()
fireworks_client = Fireworks(api_key=fireworks_key)

# Define model names
mixtral="accounts/fireworks/models/mixtral-8x22b-instruct"
llama3 ="accounts/fireworks/models/llama-v3-70b-instruct"

# Define constants for token limit and temperature
MAX_TOKENS = 500
TEMP = 0.5

def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=MAX_TOKENS,
    temperature=TEMP,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    """
    Function to get completion from OpenAI's GPT-4 model.

    Args:
        messages (list[dict[str, str]]): List of message objects.
        model (str, optional): Model name. Defaults to "gpt-4".
        max_tokens (int, optional): Maximum number of tokens in the output. Defaults to MAX_TOKENS.
        temperature (float, optional): Sampling temperature. Defaults to TEMP.
        stop ([type], optional): Sequence of tokens to stop at. Defaults to None.
        seed (int, optional): Random seed for deterministic output. Defaults to 123.
        tools ([type], optional): Tools to use. Defaults to None.
        logprobs ([type], optional): Whether to return log probabilities of the output tokens. Defaults to None.
        top_logprobs ([type], optional): Number of top log probabilities to return. Defaults to None.

    Returns:
        str: Completion from the model.
    """
    # Prepare parameters for the API call
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }

    # Add tools to parameters if provided
    if tools:
        params["tools"] = tools

    # Call the API and get the completion
    completion = client.chat.completions.create(**params)
    return completion

def get_fireworks_completion(messages, fireworks_model=mixtral):
    """
    Function to get completion from Fireworks' model.

    Args:
        messages (list[dict[str, str]]): List of message objects.
        fireworks_model (str, optional): Model name. Defaults to mixtral.

    Returns:
        [type]: Completion from the model.
    """
    # Call the API and get the completion
    chat_completion = fireworks_client.chat.completions.create(
        model = fireworks_model,
        messages = messages,
        max_tokens=MAX_TOKENS,
        temperature=TEMP,   
    )
    return chat_completion

def get_anthropic_completion(messages):
    """
    Function to get completion from Anthropics' model.

    Args:
        messages (list[dict[str, str]]): List of message objects.

    Returns:
        [type]: Completion from the model.
    """
    # Call the API and get the completion
    chat_completion = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=MAX_TOKENS,
        temperature=TEMP,
        messages=messages
    )   
    return chat_completion

# Load data from CSV file
df = pd.read_csv("Chat_selection.csv")

# Filter rows where Context is "Relationship"
df_rel = df[df["Context"] == "Relationship"]


In [12]:
def get_prompt_victim(entry):
    """
    Function to generate a prompt for giving advice to the manipulated person based on the context of the conversation.

    Args:
        entry (dict): A dictionary containing details of the conversation.

    Returns:
        str: A formatted string that serves as the promot to generate advice for the manipulated person.
    """
    # Check the context of the conversation and generate the appropriate prompt
    if entry["Context"] == "Workplace-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my boss ({entry["Perp"]}):
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""

    elif entry["Context"] == "workplace-no-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my colleagues:
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""
    
    elif entry["Context"] == "friendship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my friends:
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""

    elif entry["Context"] == "Relationship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my romantic partner:
        {entry["Chat"]}
        The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?"""

def get_prompt_culprit(entry):
    """
    Function to generate a prompt to give advice to the manipulating person based on the context of the conversation.

    Args:
        entry (dict): A dictionary containing details of the conversation.

    Returns:
        str: A formatted string that serves as the prompt for advice for the manipulating person.
    """
    # Check the context of the conversation and generate the appropriate prompt
    if entry["Context"] == "Workplace-hierarchy":
        return f"""Below is an email conversation between me ({entry["Perp"]}) and my employees:
        {entry["Chat"]}
        I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""

    elif entry["Context"] == "workplace-no-hierarchy":
        return f"""Below is an email conversation between me ({entry["Perp"]}) and my colleagues:
        {entry["Chat"]}
        I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""
    
    elif entry["Context"] == "friendship":
        return f"""Below is a Whatsapp conversation between me ({entry["Perp"]}) and my friends:
        {entry["Chat"]}
         I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""
    
    elif entry["Context"] == "Relationship":
        return f"""Below is a Whatsapp conversation between me ({entry["Perp"]}) and my romantic partner:
        {entry["Chat"]}
         I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?"""
        
def get_prompt_what_next(entry):
    """
    Function to generate a prompt for the next action based on the context of the conversation.

    Args:
        entry (dict): A dictionary containing details of the conversation.

    Returns:
        str: A formatted string that serves as the prompt for the next action.
    """
    # Check the context of the conversation and generate the appropriate prompt
    if entry["Context"] == "Workplace-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my boss ({entry["Perp"]}):
        {entry["Chat"]}
        What should I say next? Why would that be the right reaction?"""

    elif entry["Context"] == "workplace-no-hierarchy":
        return f"""Below is an email conversation between me ({entry["Victim"]}) and my colleagues:
        {entry["Chat"]}
         What should I say next? Why would that be the right reaction?"""
    
    elif entry["Context"] == "friendship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my friends:
        {entry["Chat"]}
         What should I say next? Why would that be the right reaction?"""

    elif entry["Context"] == "Relationship":
        return f"""Below is a Whatsapp conversation between me ({entry["Victim"]}) and my romantic partner:
        {entry["Chat"]}
         What should I say next? Why would that be the right reaction?"""

In [13]:
responses = {}
# Iterate over each row in the DataFrame
for i, entry in df.iterrows():
    responses[i] = {}

    # Generate prompts for the victim, culprit, and what's next based on the entry
    message_victim = [{
        'role': 'user',
        'content': get_prompt_victim(entry),
      },
    ]

    message_culprit = [{
        'role': 'user',
        'content': get_prompt_culprit(entry),
      },
    ]

    message_what_next = [{
        'role': 'user',
        'content': get_prompt_what_next(entry),
      },
    ]

    # Get responses from different models for the victim
    anthropic_response = get_anthropic_completion(messages=message_victim)
    gpt_4_response = get_completion(messages=message_victim)
    mixtral_response = get_fireworks_completion(messages=message_victim, fireworks_model=mixtral)
    llama3_response = get_fireworks_completion(messages=message_victim, fireworks_model=llama3)

    # Store the chat details and responses in the responses dictionary
    responses[i]["chat"] = entry["Chat"]
    responses[i]["context"] = entry["Context"]
    responses[i]["Victim"] = entry["Victim"]
    responses[i]["culprit"] = entry["Perp"]
    responses[i]["manipulation_strategy"] = entry["Strategy"]

    responses[i]["mixtral_instruct_response_victim"] = mixtral_response.choices[0].message.content
    responses[i]["llama3_instruct_respons_victim"] = llama3_response.choices[0].message.content
    responses[i]["gpt_4_response_victim"] = gpt_4_response.choices[0].message.content
    responses[i]["claude_opus_response_victim"] = anthropic_response.content[0].text

    # Get responses from different models for the culprit
    anthropic_response_culprit = get_anthropic_completion(messages=message_culprit)
    gpt_4_response_culprit = get_completion(messages=message_culprit)
    mixtral_response_culprit = get_fireworks_completion(messages=message_culprit, fireworks_model=mixtral)
    llama3_response_culprit = get_fireworks_completion(messages=message_culprit, fireworks_model=llama3)

    responses[i]["gpt_4_response_culprit"] = gpt_4_response_culprit.choices[0].message.content
    responses[i]["claude_opus_response_culprit"] = anthropic_response_culprit.content[0].text
    responses[i]["mixtral_instruct_response_culprit"] = mixtral_response_culprit.choices[0].message.content
    responses[i]["llama3_instruct_respons_culprit"] = llama3_response_culprit.choices[0].message.content

    # Get responses from different models for what's next
    anthropic_response_what_next = get_anthropic_completion(messages=message_what_next)
    gpt_4_response_what_next = get_completion(messages=message_what_next)
    mixtral_response_what_next = get_fireworks_completion(messages=message_what_next, fireworks_model=mixtral)
    llama3_response_what_next = get_fireworks_completion(messages=message_what_next, fireworks_model=llama3)

    responses[i]["gpt_4_response_what_next"] = gpt_4_response_what_next.choices[0].message.content
    responses[i]["claude_opus_response_what_next"] = anthropic_response_what_next.content[0].text
    responses[i]["mixtral_instruct_response_what_next"] = mixtral_response_what_next.choices[0].message.content
    responses[i]["llama3_instruct_respons_what_next"] = llama3_response_what_next.choices[0].message.content

    # Convert the responses dictionary to a DataFrame and save it to a CSV file
    df = pd.DataFrame.from_dict(responses, orient='index')
    df.to_csv("generated_advice.csv")